# Vacationpy Analysis

In [1]:
# import dependencies

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#importing weather data from weatherPy exported csv
weather_df = pd.read_csv("weather_data.csv")
weather_df


,City,country,Date,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind Speed
0,Mataura,NZ,1650139304,-46.1927,168.8643,276.75,96,68,1.22
1,Cidreira,BR,1650139304,-30.1811,-50.2056,292.73,68,100,3.42
2,Novy Urengoy,RU,1650139121,66.0833,76.6333,264.11,92,100,7.57
3,Dikson,RU,1650139305,73.5069,80.5464,261.80,94,100,7.26
4,Juan Lacaze,UY,1650139305,-34.4342,-57.4381,293.21,65,29,5.74
...,...,...,...,...,...,...,...,...,...
543,Ambanja,MG,1650139527,-13.6833,48.4500,297.97,90,19,1.33
544,Talara,PE,1650139528,-4.5772,-81.2719,300.57,54,0,6.69
545,Jijiga,ET,1650139528,9.3500,42.8000,293.92,65,96,3.04
546,Carauari,BR,1650139529,-4.8828,-66.8958,303.71,61,94,0.72


# Creating Humidity heatmap

In [3]:
#configure gmap with api key
gmaps.configure(api_key = g_key)


In [4]:
#storing latitude and logitude as locations
locations = weather_df[["Latitude", "Longitude"]].astype(float)

#getting humidity data
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()

max_humidity

100.0

In [5]:
#creating humidity heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False,
                                 max_intensity = max_humidity, point_radius = 3)
fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))

# Creating new dataframe with narrowed down criteria


In [6]:
#limit max temp to a range of 70F (294 K) to 85F (302 K)
#limit windspeed < 12
#limit cloudiness <30
weather_pref = weather_df[(weather_df["Max_temp"]>=294) & (weather_df["Max_temp"]<=302)
                             & (weather_df["Wind Speed"]<=12)& (weather_df["Cloudiness"]<=20)]

print(weather_pref.shape)
weather_pref.head()


(54, 9)


,City,country,Date,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind Speed
10,Kununurra,AU,1650139120,-15.7667,128.7333,297.14,94,19,2.06
36,Rikitea,PF,1650139318,-23.1203,-134.9692,298.76,71,4,6.94
41,Pitimbu,BR,1650139321,-7.4706,-34.8086,301.16,77,6,3.93
43,Ambilobe,MG,1650139322,-13.2000,49.0500,296.19,87,10,1.92
61,George Town,MY,1650139329,5.4112,100.3354,300.11,89,20,0.00


In [7]:
#checking for any null values
weather_pref.isnull().values.any()

False

# Hotel Map


In [8]:
#Creating new dataframe for hotel map
hotel_df = weather_pref[["City", "country", "Latitude", "Longitude"]]
hotel_df = hotel_df.reset_index()
hotel_df["Hotel Name"] = " "
hotel_df.head()

,index,City,country,Latitude,Longitude,Hotel Name
0,10,Kununurra,AU,-15.7667,128.7333,
1,36,Rikitea,PF,-23.1203,-134.9692,
2,41,Pitimbu,BR,-7.4706,-34.8086,
3,43,Ambilobe,MG,-13.2000,49.0500,
4,61,George Town,MY,5.4112,100.3354,


In [12]:
# using google places API to find nearest hotels

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
cities = hotel_df["City"]
params = {"location": f"0,0",  
    "radius": "5000",
    "type": "lodging",
    "key": g_key}
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    

    params["location"] = f"{lat},{lng}"

    response = requests.get(base_url, params = params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except(KeyError, IndexError):
        print("hotel not found....Skipping")


#print(response)

hotel not found....Skipping
hotel not found....Skipping
hotel not found....Skipping
hotel not found....Skipping
hotel not found....Skipping
hotel not found....Skipping


In [13]:
# Check final hotel datafrme
hotel_df.head()

,index,City,country,Latitude,Longitude,Hotel Name
0,10,Kununurra,AU,-15.7667,128.7333,Hotel Kununurra
1,36,Rikitea,PF,-23.1203,-134.9692,People ThankYou
2,41,Pitimbu,BR,-7.4706,-34.8086,Reserva do Abiaí
3,43,Ambilobe,MG,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
4,61,George Town,MY,5.4112,100.3354,Cititel Penang
